In [ ]:
import sys
sys.path.append("..")

import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from reader import generic_reader
from inflation import read_cpi_by_year, convert_usd

In [ ]:
cpi_by_year = read_cpi_by_year("inflation_data/cpiai.csv")

NameError: name 'read_cpi_by_year' is not defined

## Prepartion of movie data

Our dataset of movies, are movies from the USA, from the periode 1/1/2000 to 31/12/2023 and with a vote count of atleast 10

In [ ]:
movies = generic_reader.read_csv_file_to_data_frame("movie_data/movies.csv")
print(movies.info())
movies

We make a new dataframe keeping only the columns we want to use.

In [ ]:
columns_to_keep = ['budget', 'movie_id', 'original_title', 'overview', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director_person_ids', 'genre_ids', 'collection_id', 'cast_person_ids', 'cast_credit_ids', 'crew_person_ids', 'crew_credit_ids']
df_movies = movies[columns_to_keep].copy()

df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies['genre_ids'] = movies['genre_ids'].apply(ast.literal_eval)
df_movies

Removing animation and documentary from the dataset as we'll be fociusing on motion pictures

In [ ]:
df_movies = df_movies[df_movies['genre_ids'].map({16, 99}.isdisjoint)]

In [ ]:
df_movies.describe()

Next we'll filter the runtime, dropping movies which have a runtime below 60 mins and above 3 hours

In [ ]:
df_movies = df_movies[(df_movies['runtime'] >= 60) & (df_movies['runtime'] <= 240)]

In [ ]:
short_count = (df_movies['runtime'] <= 60).sum()
long_count = (df_movies['runtime'] >= 240).sum()

print(f"Movies shorter than or equal to 60 min: {short_count}")
print(f"Movies longer than or equal to 180 min: {long_count}")


[placerholder text] Converting to USD 2023

In [ ]:
df_movies['budget_2023_usd'] = df_movies.apply(
    lambda row: convert_usd(
        original_amount=row['budget'],
        original_year=row['release_date'].year,
        new_year=2023,
        cpi_by_year=cpi_by_year
    ),
    axis=1
)

# Convert revenue to 2023 USD
df_movies['revenue_2023_usd'] = df_movies.apply(
    lambda row: convert_usd(
        original_amount=row['revenue'],
        original_year=row['release_date'].year,
        new_year=2023,
        cpi_by_year=cpi_by_year
    ),
    axis=1
)

In [ ]:
persons = generic_reader.read_csv_file_to_data_frame("movie_data/persons.csv")

In [ ]:
persons.info()

In [ ]:
columns_to_keep = ['budget', 'movie_id', 'original_title', 'overview', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director_person_ids', 'genre_ids', 'collection_id', 'cast_person_ids', 'cast_credit_ids', 'crew_person_ids', 'crew_credit_ids']
df_persons = persons[columns_to_keep].copy()

df_persons['genre_ids'] = persons['genre_ids'].apply(ast.literal_eval)
df_persons